# testing stuff

In [1]:
import cv2
import matplotlib.pyplot as plt  
# iscrtavanje slika i plotova unutar samog browsera
%matplotlib inline 

import matplotlib.pylab as pylab
# prikaz vecih slika 
pylab.rcParams['figure.figsize'] = 21,15

In [3]:
for i in range(5):
    print 'a',
    i +=3

a a a a a


In [5]:
import os
os.makedirs('./images/kernel_tests/x')
# os.makedirs('./images/kernel_tests/x') # WindowsError: [Error 183] Cannot .. exists: './images/kernel_tests/x'

WindowsError: [Error 183] Cannot create a file when that file already exists: './images/kernel_tests/x'

In [12]:
######
# x je niz od onoliko crnih piksela koliko je prosjecna debljina linije (+-1 ili 2)
# x = [0] * (int(thickness)-1)
# y je niz od onoliko crnih piksela koliko je prosjecna debljina linije + 2 ili 3 ||| INT!
#   ~ pikseli iz y niza pocinju od posljednje tacke posmatrane linije (tj.kandidata za liniju)
# y = current_pos + int(spacing * 0.9 ili 0.8)
# print x_in_y(x, y)
###
def x_in_y(x, y):
    try:
        x_len = len(x)
    except TypeError:
        x_len = 1
        x = type(y)((x,))

    for i in xrange(len(y)):
        if (y[i : i+x_len] == x):
            return True
    return False

In [11]:
def locate_lines(img):
    '''Locate lines.
    Returns: staff lines LOCATIONS and the lines-only image'''
    threshold = cv2.ADAPTIVE_THRESH_MEAN_C
    method, block, c = cv2.THRESH_BINARY, 51, 35
    img_ada = cv2.adaptiveThreshold(img, 255, threshold, method, block, c)

    kernel_w, kernel_h = 1,30
    kernel = np.ones((kernel_w, kernel_h), dtype=np.int)
    lines_only_img = cv2.dilate(img_ada, kernel, iterations=1)
    
img = cv2.imread('./images/dataset/muzikanti.JPG', 0) # 0 --> read as grayscale
# locate_lines(img)

In [35]:
##### ## ## ## ## ## ## ##
# Test of a special kernel for line reconstruction (and maybe removal of non-line symbols)
### ## ## ## ## ## ## ##
# Create two kernels. One with a `0` on the X spot, and one with a `1`.
# [
#  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
#  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
#  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
#  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
#  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
#  [0, 0, 0, 0, 0, X, 0, 0, 0, 0, 0],
#  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
#  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
#  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
#  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
#  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
# ]
kernel0 = np.zeros((11, 11), np.uint8)
kernel0[0] = [1]*11
kernel0[10] = [1]*11

kernel1 = np.zeros((11, 11), np.uint8)
kernel1[0] = [1]*11
kernel1[10] = [1]*11
kernel1[5][5] = 1 # the only difference between `kernel0` and `kernel1`

img = cv2.imread('./images/my_kernel/my_kernel.jpg', 0) # as grayscale
# img = cv2.threshold(img, 127, 255, cv2.THRESH_BINARY)
ret, image_bin = cv2.threshold(img, 180, 255, cv2.THRESH_BINARY) # ret je vrednost praga, image_bin je binarna slika
# plt.imshow(img_bin, 'gray')
print image_bin.dtype
img_er0 = cv2.erode(image_bin, kernel0)
cv2.imwrite('./images/my_kernel/E_0.jpg', img_er0)
img_er1 = cv2.erode(image_bin, kernel1)
cv2.imwrite('./images/my_kernel/E_1.jpg', img_er1)
img_dil0 = cv2.dilate(image_bin, kernel0)
cv2.imwrite('./images/my_kernel/D_0.jpg', img_dil0)
img_dil1 = cv2.dilate(image_bin, kernel1)
cv2.imwrite('./images/my_kernel/D_1.jpg', img_dil1)

uint8


True

In [4]:
img = cv2.imread('./images/dataset/viva1_up.jpg')
img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) # konvertovanje iz BGR u GRAYSCALE
# plt.imshow(img, 'gray')  # prikazivanje grayscale slike
cv2.imwrite('./images/dataset/viva_gray.jpg', img)

True